# Mask R-CNN - Inspect Bottle Trained Model

Code and visualizations to test, debug, and evaluate the Mask R-CNN model.

In [1]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Root directory of the project
ROOT_DIR = os.path.abspath("../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log

import bottle

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

BOTTLE_WEIGHTS_PATH = os.path.join(MODEL_DIR,"mask_rcnn_bottle_0100.h5")  # TODO: update this path

Using TensorFlow backend.


/home/jaikumar/FinalMaskRCNN/git-repo/Mask-R-CNN/bottle


## Configurations

In [2]:
config = bottle.BottleConfig()
BOTTLE_DIR = os.path.join(ROOT_DIR, "dataset")

In [3]:
# Override the training configurations with a few
# changes for inferencing.
class InferenceConfig(config.__class__):
    # Run detection on one image at a time
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.9
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

## Notebook Preferences

In [4]:
# Device to load the neural network on.
# Useful if you're training a model on the same 
# machine, in which case use CPU and leave the
# GPU for training.
DEVICE = "/gpu:0"  # /cpu:0 or /gpu:0

# Inspect the model in training or inference modes
# values: 'inference' or 'training'
# TODO: code for 'training' test mode not ready yet
TEST_MODE = "inference"

In [5]:
def get_ax(rows=1, cols=1, size=16):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Adjust the size attribute to control how big to render images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

## Load Validation Dataset

In [6]:
# Load validation dataset
dataset = bottle.BottleDataset()
dataset.load_bottle(BOTTLE_DIR, "val")

dataset_tr = bottle.BottleDataset()
dataset_tr.load_bottle(BOTTLE_DIR, "train")

dataset_tr.prepare()

# Must call before using the dataset
dataset.prepare()
print("Images: {}\nClasses: {}".format(len(dataset_tr.image_ids), dataset_tr.class_names))
print("Images: {}\nClasses: {}".format(len(dataset.image_ids), dataset.class_names))

Images: 177
Classes: ['BG', 'bottle']
Images: 15
Classes: ['BG', 'bottle']


## Load Model

In [7]:
# Create model in inference mode
with tf.device(DEVICE):
    model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR,
                              config=config)







Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
box_ind is deprecated, use box_indices instead


Instructions for updating:
Use `tf.cast` instead.


In [8]:
# Set path to bottle weights file

# Download file from the Releases page and set its path
# https://github.com/matterport/Mask_RCNN/releases
# weights_path = "/path/to/mask_rcnn_bottle.h5"

# Or, load the last model you trained
#weights_path = model.find_last()

# Load weights
#print("Loading weights ", weights_path)
model.load_weights(BOTTLE_WEIGHTS_PATH, by_name=True)

## Run Detection

In [ ]:
### To Download video from YouTube
#import pytube
#from pytube import YouTube
#import os
#def downloadYouTube(videourl, path):
#    yt = YouTube(videourl)
#    yt = yt.streams.filter(progressive=True, file_extension='mp4').order_by('resolution').desc().first()
#    if not os.path.exists(path):
#        os.makedirs(path)
#        yt.download(path)
#downloadYouTube('https://youtu.be/', './videos')###

In [1]:
#VIDEO PREDICTIION mp4 format

In [9]:
import cv2
import numpy as np
import os

def random_colors(N):
    np.random.seed(1)
    colors = [tuple(255 * np.random.rand(3)) for _ in range(N)]
    return colors


def apply_mask(image, mask, color, alpha=0.5):
    """apply mask to image"""
    for n, c in enumerate(color):
        image[:, :, n] = np.where(
            mask == 1,
            image[:, :, n] * (1 - alpha) + alpha * c,
            image[:, :, n]
        )
    return image

def display_instances(image, boxes, masks, ids, names, scores):
    """
        take the image and results and apply the mask, box, and Label
    """
    n_instances = boxes.shape[0]
    colors = random_colors(n_instances)

    if not n_instances:
        print('NO INSTANCES TO DISPLAY')
    else:
        assert boxes.shape[0] == masks.shape[-1] == ids.shape[0]

    for i, color in enumerate(colors):
        if not np.any(boxes[i]):
            continue

        y1, x1, y2, x2 = boxes[i]
        label = names[ids[i]]
        score = scores[i] if scores is not None else None
        caption = '{} {:.2f}'.format(label, score) if score else label
        mask = masks[:, :, i]

        image = apply_mask(image, mask, color)
        image = cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
        image = cv2.putText(
            image, caption, (x1, y1), cv2.FONT_HERSHEY_COMPLEX, 0.7, color, 2
        )

    return image

In [10]:
import cv2
if __name__ == '__main__' :

    #Mask R-CNN
    ROOT_DIR = os.path.abspath("../")
    MODEL_DIR = os.path.join(ROOT_DIR,"logs")
    VIDEO_DIR = os.path.join(ROOT_DIR, "output")
    VIDEO_SAVE_DIR = os.path.join(VIDEO_DIR, "savedimgs")
    COCO_MODEL_PATH = os.path.join(MODEL_DIR, "mask_rcnn_bottle_0100.h5")
    if not os.path.exists(COCO_MODEL_PATH):
     utils.download_trained_weights(COCO_MODEL_PATH)
     class InferenceConfig(bottle.BottleConfig):
         GPU_COUNT = 1
         batch_size = 3
         IMAGES_PER_GPU = batch_size
         config = InferenceConfig()
         config.display()
        
# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR,
                              config=config)

#model = modellib.MaskRCNN(mode=”inference”, model_dir=MODEL_DIR, config=config)
# Load weights trained on MS-COCO
model.load_weights(COCO_MODEL_PATH, by_name=True)
class_names = ['BG','bottle']    
video = cv2.VideoCapture(os.path.join(ROOT_DIR,"videos/plasticbottle.mov"));

# Find OpenCV version
(major_ver, minor_ver, subminor_ver) = (cv2.__version__).split('.')

if int(major_ver)  < 3 :
    fps = video.get(cv2.cv.CV_CAP_PROP_FPS)
    print ("Frames per second using video.get(cv2.cv.CV_CAP_PROP_FPS): {0}".format(fps))
else :
    fps = video.get(cv2.CAP_PROP_FPS)
    print ("Frames per second using video.get(cv2.CAP_PROP_FPS) : {0}".format(fps))

try:
    if not os.path.exists(VIDEO_SAVE_DIR): 
        os.makedirs(VIDEO_SAVE_DIR)
except OSError:
    print ('Error: Creating directory of data')
                
frames = []
frame_count = 0
print(" Segmentation in progress....")
while True:
    ret, frame = video.read()
    if not ret: 
        break
 # Save each frame of the video to a list
    frame_count += 1
    frames.append(frame)
    ##print('frame_count :{0}'.format(frame_count))
    batch_size=1
    BATCH_SIZE = 1
    if len(frames) == BATCH_SIZE:
        results = model.detect(frames, verbose=0)
        #print('Predicted')
        for i, item in enumerate(zip(frames, results)):
            frame = item[0]
            r = item[1]
            frame = display_instances(
            frame, r['rois'], r['masks'], r['class_ids'], class_names, r['scores'])
            name = '{0}.jpg'.format(frame_count + i - batch_size)
            name = os.path.join(VIDEO_SAVE_DIR, name)
            cv2.imwrite(name, frame)
            ##print('writing to file:{0}'.format(name))
            # Clear the frames array to start the next batch
            frames = []  
#capture.release()
video.release(); 
print("... Segmentation Completed!")

Frames per second using video.get(cv2.CAP_PROP_FPS) : 25.0
 Segmentation in progress....
... Segmentation Completed!


In [14]:
def make_video(outvid, images=None, fps=30, size=None,
               is_color=True, format="FMP4"):
    from cv2 import VideoWriter, VideoWriter_fourcc, imread, resize
    fourcc = VideoWriter_fourcc(*format)
    vid = None
    for image in images:
        if not os.path.exists(image):
            raise FileNotFoundError(image)
        img = imread(image)
        if vid is None:
            if size is None:
                size = img.shape[1], img.shape[0]
            vid = VideoWriter(outvid, fourcc, float(fps), size, is_color)
        if size[0] != img.shape[1] and size[1] != img.shape[0]:
            img = resize(img, size)
        vid.write(img)
    vid.release()
    return vid

import glob
import os
# Directory of images to run detection on
#ROOT_DIR = os.getcwd()
OUT_DIR = os.path.join(ROOT_DIR, "output")
VIDEO_SAVE_DIR = os.path.join(VIDEO_DIR, "savedimgs")
images = list(glob.iglob(os.path.join(VIDEO_SAVE_DIR, '*.*')))
# Sort the images by integer index
images = sorted(images, key=lambda x: float(os.path.split(x)[1][:-3]))

outvid = os.path.join(OUT_DIR, "videosegment.mp4")
make_video(outvid, images, fps=30)

<VideoWriter 0x7f6e4f214290>

In [ ]:
!ls -alh ../output/